In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import gurobipy as gb
import os
from pickle import dump, load
from time import process_time
from copy import deepcopy
from source import*

path = os.getcwd()

In [ ]:
file = open(path+"/Results/Configurations/Open Stations/open_stations_18","rb"); S = load(file); file.close()
K_s = {}                # {scenario:{station:[vehicles]}}\n",
K = {}                  # {scenatio:[vehicles]}\n",

for sc in range(25):
    file = open(path+f"/Data/K/K_sc{sc}", "rb")
    K[sc] = load(file); file.close()

    file = open(path+f"/Data/K_s10/Ks_sc{sc}", "rb")
    K_s[sc] = load(file); file.close()

original_stress = {}
for s in S:
    original_stress[s] = np.mean(np.array([len(K_s[sc][s]) for sc in range(25)]))

max_s = max(original_stress, key=original_stress.get)
max_s, original_stress[max_s]"

In [ ]:
"def generate_routes_given_number_chargers(s,S,y,K,K_s_prime,exhaust=False):\n",
<<<<<<< HEAD
    "    \n",
=======
>>>>>>> 3a86e469018b894faf389038ee08ed8d347d381b
    "    total_serviced_costumers:dict[list[list]] = dict()\n",
    "\n",
    "    for sc in range(25):\n",
    "        routes = list()\n",
    "        K, _, S_k, a, t = load_pickle(path+\"/Data/\",sc)\n",
    "        K_s = K_s_prime[sc]\n",
<<<<<<< HEAD
    "        S_k = {k:[ss for ss in S_k[k] if ss in S] for k in K_s[s]}\n",
    "        a.update({(\"s\",s):0}); a.update({(\"e\",s):40})\n",
    "        t.update({(\"s\",s):0}); t.update({(\"e\",s):0})\n",
    "\n",
    "        mp = gb.Model(\"Restricted Master Problem\")\n",
    "\n",
    "        dummy_0 = {k:mp.addVar(vtype=gb.GRB.CONTINUOUS, obj=1, name=f\"st0_{k}\") for k in K_s[s]}\n",
    "        aux = mp.addVar(vtype=gb.GRB.CONTINUOUS, obj=2, name=f\"aux_{s}\")\n",
    "\n",
    "        vehic_assign = {}\n",
    "        for k in K_s[s]:\n",
    "            vehic_assign[k] = mp.addLConstr(dummy_0[k] == 1, name=f\"V{k}_assignment\")\n",
    "\n",
    "        st_conv = mp.addConstr(aux <= y, f\"S{s}_convexity\")\n",
    "\n",
    "        mp.setParam(\"OutputFlag\",0)\n",
    "\n",
    "        time0 = process_time(); i = 0\n",
    "        lbd = []; lastMP = 1e3\n",
    "\n",
    "        while True:\n",
    "            \n",
    "            mp.update()\n",
    "            mp.optimize()\n",
    "\n",
    "            pi = {k:vehic_assign[k].getAttr(\"Pi\") for k in K_s[s]}\n",
    "            sigma = st_conv.getAttr(\"Pi\")\n",
    "            thisMP = mp.getObjective().getValue()\n",
    "\n",
    "            if (lastMP-thisMP < 0.1 and thisMP > 5) or (lastMP-thisMP < 0.01 and thisMP<=5):\n",
    "                #if y == 8: print(f\"\\t\\tOut of Labeling scenario {sc}\")\n",
    "                mpsol = thisMP; break\n",
    "\n",
    "            #print(f\"\\tIteration {i}:\\t\\tMP obj: {round(mp.getObjective().getValue(),2)}\\ttime: {round(process_time()-time0,2)}s\")\n",
    "            i += 1\n",
    "\n",
    "            V,A,rc = get_graph_chargers(s,K_s[s],a,pi,sigma)\n",
    "            ext = vertices_extensions(V,A)\n",
    "            \n",
    "            #if y == 8:\n",
    "            #    print(f\"\\t\\tIteration {i}. Labeling for scenario {sc}\")\n",
    "\n",
    "            routes_DFS = {\"s\":([0],0)}; covered_nodes = {v:0 for v in V}; best_len = {v:0 for v in V}; best_t = {v:0 for v in V}; best_h = {v:30 for v in V}; num_improv = {v:0 for v in V}; tried = {v:0 for v in V}\n",
    "            marked = {}; max_successors = {v:100 for v in V}\n",
    "            mod_label_DFS(v=\"s\",rP=0,tP=0,qP=0,hP=0,P=[],cK=covered_nodes,L=routes_DFS,s=s,r=rc,t=t,a=a,ext=ext,best_len=best_len,best_t=best_t,best_h=best_h,tried=tried,marked=marked,num_improv=num_improv,max_successors=max_successors)\n",
    "            del routes_DFS[\"s\"]\n",
    "            \n",
    "            opt = 0\n",
    "            for l in routes_DFS:\n",
    "                if routes_DFS[l][1] < -0.001:\n",
    "                    col = {k:1 if k in routes_DFS[l][0] else 0 for k in K_s[s]}\n",
    "                    new_Col = gb.Column(col.values(),vehic_assign.values())\n",
    "                    lbd.append(mp.addVar(vtype=gb.GRB.CONTINUOUS,obj=0,column=new_Col))\n",
    "\n",
    "                    mp.chgCoeff(st_conv,lbd[-1],1)\n",
    "                    opt +=1\n",
    "            if i == 1:\n",
    "                original_routes = routes_DFS\n",
    "\n",
    "            if opt == 0 : mpsol = thisMP; break\n",
    "            lastMP = thisMP\n",
    "        \n",
    "        for v in mp.getVars():\n",
    "            v.vtype = gb.GRB.BINARY\n",
    "        #if mpsol < len(K_s[s]): mp.setParam(\"MIPGap\",0.01)\n",
    "        mp.setParam(\"MIPFocus\",2)\n",
    "        mp.update(); mp.optimize()\n",
    "\n",
    "        print(f\"\\t\\tSolved IMP scenario {sc}, with {mp.getObjective().getValue()} unnasigned vehicles\")\n",
    "        # print(f\"IMP solution: {mp.getObjective().getValue()} assigned vehicles with {y} chargers\")\n",
    "        for l in lbd:\n",
    "            if l.X > 0.5:\n",
    "                col = [k for k in K_s[s] if mp.getCoeff(vehic_assign[k],l) == 1]\n",
    "                routes.append(col)\n",
    "\n",
    "        total_serviced_costumers[sc]:list = list()\n",
    "\n",
    "        for route in routes:\n",
    "            total_serviced_costumers[sc].extend(route)\n",
    "        \n",
    "        if len(total_serviced_costumers[sc]) < len(K_s_prime[sc][s]) and y < 8:\n",
    "            print(f'\\t{y} number of chargers is infeasible\\n')\n",
    "            return None\n",
    "    \n",
    "    return total_serviced_costumers"